## CNN Unet Model with Distance as additional Feature


In [46]:
import imp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import os

sys.path.insert(0, '../../src')

from distance import coastline_dist_lat, coastline_dist_lon
from utils import df_to_xarray,read_xarray, custom_rmse


sys.path.insert(0, '../../src/preprocess')
from data_preprocess import preprocess_images,inverse_scale_image, preprocess_image_reduced

In [47]:
dir_name = "../../data/data1"
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2 = read_xarray(dir_name)

lat=coastline_dist_lat(pco2t2.pCO2.data[0])
lon=coastline_dist_lon(pco2t2.pCO2.data[0])

lat = np.repeat(lat[np.newaxis, :, : ], 421, axis=0)
lon = np.repeat(lon[np.newaxis, :, :], 421, axis=0)


In [48]:

chl_images = preprocess_image_reduced(chl.Chl.data)
mld_images = preprocess_image_reduced(mld.MLD.data)
sss_images = preprocess_image_reduced(sss.SSS.data)
sst_images = preprocess_image_reduced(sst.SST.data)
xco2_images = preprocess_image_reduced(xco2.XCO2.data,xco2=True)
pco2_images = preprocess_image_reduced(pco2.pCO2.data)
    
X = np.dstack((chl_images, mld_images, sss_images, sst_images, xco2_images,lat,lon))
X = X.reshape((421,180,360,7),order='F')

INPUT_SHAPE=X[0].shape
OUTPUT_SHAPE=pco2_images[0].shape

INPUT_SHAPE

(180, 360, 7)

In [59]:
tf.keras.backend.clear_session()

In [60]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=5,activation='elu',
                        padding="SAME")


base_model = keras.models.Sequential([
    DefaultConv2D(filters=32, input_shape=INPUT_SHAPE),
    DefaultConv2D(filters=32),
    keras.layers.MaxPooling2D(pool_size=3),
    keras.layers.Dropout(0.2),

    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    keras.layers.MaxPooling2D(pool_size=3),
    keras.layers.Dropout(0.2),

    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),

    keras.layers.UpSampling2D(size=3),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    keras.layers.Dropout(0.2),

    keras.layers.UpSampling2D(size=3),
    DefaultConv2D(filters=32),
    DefaultConv2D(filters=2),
    DefaultConv2D(filters=1,kernel_size=1),
    keras.layers.Reshape(OUTPUT_SHAPE)
   
])



base_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 180, 360, 32)      5632      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 180, 360, 32)      25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 60, 120, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 60, 120, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 120, 64)       51264     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 120, 64)       102464    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 40, 64)        0

In [61]:
myLearnRate=0.0005
custom_opt = tf.keras.optimizers.Adam(learning_rate=myLearnRate)
base_model.compile(loss=custom_rmse, optimizer=custom_opt)

In [62]:
model_path="../models/base_model/u_net_dist.h5"
early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

history = base_model.fit(X,pco2_images, epochs=500, 
                         validation_data=(X,pco2_images),
                         workers=-1,batch_size=16,
                         callbacks=callbacks)


Epoch 1/500
27/27 [==============================] - 6s 187ms/step - loss: 57.9337 - val_loss: 24.4580
Epoch 2/500
27/27 [==============================] - 4s 153ms/step - loss: 19.1673 - val_loss: 19.6090
Epoch 3/500
27/27 [==============================] - 4s 153ms/step - loss: 14.5098 - val_loss: 14.6675
Epoch 4/500
27/27 [==============================] - 4s 153ms/step - loss: 13.1159 - val_loss: 14.4117
Epoch 5/500
27/27 [==============================] - 4s 153ms/step - loss: 12.0491 - val_loss: 15.1454
Epoch 6/500
27/27 [==============================] - 4s 153ms/step - loss: 11.3764 - val_loss: 16.4364
Epoch 7/500
27/27 [==============================] - 4s 154ms/step - loss: 10.6399 - val_loss: 11.5286
Epoch 8/500
27/27 [==============================] - 4s 153ms/step - loss: 9.4865 - val_loss: 10.9090
Epoch 9/500
27/27 [==============================] - 4s 153ms/step - loss: 9.1192 - val_loss: 13.0229
Epoch 10/500
27/27 [==============================] - 4s 153ms/step - loss: